In [9]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Список символов, включая символы татарского алфавита
tatar_symbols = [
    'а', 'ә', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м',
    'н', 'о', 'ө', 'п', 'р', 'с', 'т', 'у', 'ү', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', ' ', '!', '?'
]

# Примерная функция для преобразования текста в последовательность символов
def text_to_sequence(text):
    return [tatar_symbols.index(char) for char in text if char in tatar_symbols]

# Загрузка данных
data = pd.read_csv("hack/dataset.csv")

# Предположим, что функция `text_to_sequence` уже реализована
data['text_encoded'] = data['text'].apply(text_to_sequence)

# Разделение данных
train_data, test_data = train_test_split(data, test_size=0.2)

# Паддинг для текстовых последовательностей
train_text = pad_sequences(train_data['text_encoded'], maxlen=100, padding='post')
test_text = pad_sequences(test_data['text_encoded'], maxlen=100, padding='post')


In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, LSTM, Dense, TimeDistributed

def build_model(input_dim, output_dim):
    # Входной слой
    input_layer = Input(shape=(None, input_dim))

    # Сверточные слои для извлечения признаков
    x = Conv1D(128, kernel_size=5, strides=1, padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv1D(128, kernel_size=5, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # LSTM слой для обработки временных зависимостей
    x = LSTM(256, return_sequences=True)(x)

    # Выходной слой для предсказания мел-спектрограммы
    output_layer = TimeDistributed(Dense(output_dim))(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Определите размеры входных и выходных данных
input_dim = 100  # Длина текстовой последовательности
output_dim = 80  # Размерность мел-спектрограммы

model = build_model(input_dim, output_dim)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, 100)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, None, 128)      │        64,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, None, 128)      │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, None, 256)      │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, None, 80)       │        20,560 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 562,000 (2.14 MB)

 Trainable params: 561,488 (2.14 MB)

 Non-trainable params: 512 (2.00 KB)

In [11]:
import pandas as pd

# Загрузка данных из CSV файла
data = pd.read_csv('hack/dataset.csv')
print(data.head())


                          path  \
0  hack\ans10001_982944754.wav   
1  hack\ans10003_184648589.wav   
2  hack\ans10003_724102486.wav   
3  hack\ans10005_724102486.wav   
4  hack\ans10006_724102486.wav   

                                                text  
0  Денлоны яулап алганнан соң, Англиядә озак елла...  
1  Моның аркасында ачылышы утыз биш миллиметрлы н...  
2  Моның аркасында ачылышы утыз биш миллиметрлы н...  
3  Се шулай ук фотогеник Ма харизмадан бигрәк тыш...  
4  Сайлаулар вакытында Се Маның кризис булганда и...  


In [14]:
import librosa
import numpy as np

# Функция для загрузки и преобразования аудио в мел-спектрограмму
def load_and_convert_to_mel(path, n_mels=80, n_fft=2048, hop_length=512, sample_rate=22050):
    # Загрузка аудиофайла
    audio, sr = librosa.load(path, sr=sample_rate)
    # Преобразование в мел-спектрограмму с использованием именованных аргументов
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    # Преобразование мел-спектрограммы в децибелы
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

# Пример применения
data = pd.read_csv('hack/dataset.csv')[:10]
data['mel_spectrogram'] = data['path'].apply(lambda x: load_and_convert_to_mel(x))

In [15]:
def pad_mel_specs(mel_specs, max_length=1000):
    padded_mels = np.zeros((len(mel_specs), mel_specs[0].shape[0], max_length))
    for i, mel_spec in enumerate(mel_specs):
        if mel_spec.shape[1] > max_length:
            padded_mels[i, :, :max_length] = mel_spec[:, :max_length]
        else:
            padded_mels[i, :, :mel_spec.shape[1]] = mel_spec
    return padded_mels

# Подготовка данных
mels = list(data['mel_spectrogram'])
padded_mels = pad_mel_specs(mels)

# Пример использования в модели
# model.fit(x=padded_mels, y=..., ...)


In [27]:
model.compile(optimizer='adam', loss='mean_squared_error')

# Загрузка и подготовка аудиоданных в мел-спектрограммы должна быть реализована
train_mel = ...
test_mel = ...

history = model.fit(data['text'], padded_mels, validation_data=(test_text, test_mel), epochs=20, batch_size=32)


Epoch 1/20


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("functional_3_1/Cast:0", shape=(None, 1), dtype=float32). Expected shape (None, None, 80), but input has incompatible shape (None, 1)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(None, 1), dtype=string)
  • training=True
  • mask=None

In [18]:
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

   # Функция для загрузки и преобразования аудио в мел-спектрограмму
def load_and_convert_to_mel(path, n_mels=80, n_fft=2048, hop_length=512, sample_rate=22050):
   audio, sr = librosa.load(path, sr=sample_rate)
   mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
   mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
   return mel_spec_db

   # Функция для обработки текста (простая заглушка, реализуйте по необходимости)
def text_to_sequence(text):
   return [ord(char) for char in text]  # Простая конвертация в числа

   # Функция для паддинга мел-спектрограмм
def pad_mel_specs(mel_specs, max_length=1000):
   padded_mels = np.zeros((len(mel_specs), mel_specs[0].shape[0], max_length))
   for i, mel_spec in enumerate(mel_specs):
       if mel_spec.shape[1] > max_length:
           padded_mels[i, :, :max_length] = mel_spec[:, :max_length]
       else:
           padded_mels[i, :, :mel_spec.shape[1]] = mel_spec
   return padded_mels


In [20]:
   # Загрузка данных из CSV файла
data = pd.read_csv('hack/dataset.csv')[:10]
# Преобразование аудио
data['mel_spectrogram'] = data['path'].apply(lambda x: load_and_convert_to_mel(x))

In [21]:
   # Преобразование списка мел-спектрограмм в numpy массив
mels = list(data['mel_spectrogram'])
padded_mels = pad_mel_specs(mels)
data['text_encoded'] = data['text'].apply(text_to_sequence)
   


In [22]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, LSTM, Dense, TimeDistributed

def build_model(input_dim, output_dim):
       input_layer = Input(shape=(None, input_dim))

       x = Conv1D(128, kernel_size=5, strides=1, padding='same')(input_layer)
       x = BatchNormalization()(x)
       x = ReLU()(x)
       
       x = Conv1D(128, kernel_size=5, strides=1, padding='same')(x)
       x = BatchNormalization()(x)
       x = ReLU()(x)

       x = LSTM(256, return_sequences=True)(x)

       output_layer = TimeDistributed(Dense(output_dim))(x)

       model = Model(inputs=input_layer, outputs=output_layer)
       return model

input_dim = padded_mels.shape[1]  # Размерность входных данных
output_dim = padded_mels.shape[1]  # Размерность выходных данных (мел-спектрограмма)

model = build_model(input_dim, output_dim)
model.summary()

model.compile(optimizer='adam', loss='mean_squared_error')

   # Подготовка текстовых данных (если есть)
train_text = pad_sequences(data['text_encoded'], maxlen=100, padding='post')
   


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None, 80)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, None, 128)      │        51,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, None, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, None, 128)      │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, None, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, None, 256)      │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, None, 80)       │        20,560 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 549,200 (2.10 MB)

 Trainable params: 548,688 (2.09 MB)

 Non-trainable params: 512 (2.00 KB)

In [26]:
   # Подготовка данных для обучения (если есть текстовые данные)
train_data, test_data, train_mel, test_mel = train_test_split(train_text, padded_mels, test_size=0.2)

   # Обучение модели
history = model.fit(train_data, train_mel, validation_split=0.2, epochs=20, batch_size=32)


Epoch 1/20


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("functional_3_1/Cast:0", shape=(None, 100), dtype=float32). Expected shape (None, None, 80), but input has incompatible shape (None, 100)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(None, 100), dtype=int32)
  • training=True
  • mask=None

In [44]:
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, LSTM, Dense, TimeDistributed

# Load audio and convert to mel-spectrogram
def load_and_convert_to_mel(path, n_mels=80, n_fft=2048, hop_length=512, sample_rate=22050):
    audio, sr = librosa.load(path, sr=sample_rate)
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

# Convert text to sequence of character ordinals
def text_to_sequence(text):
    return [ord(char) for char in text]

# Pad mel spectrograms to ensure consistent dimensions
def pad_mel_specs(mel_specs, max_length=100):
    n_mels = mel_specs[0].shape[0]
    padded_mels = np.zeros((len(mel_specs), max_length, n_mels))  # Notice the dimension order: max_length, n_mels
    for i, mel_spec in enumerate(mel_specs):
        if mel_spec.shape[1] > max_length:
            padded_mels[i, :max_length, :] = mel_spec[:, :max_length].T  # Transpose to align time steps and features
        else:
            padded_mels[i, :mel_spec.shape[1], :] = mel_spec.T
    return padded_mels

# Load data
data = pd.read_csv('hack/dataset.csv')[:10]

# Process text and audio to sequences and mel spectrograms
data['text_encoded'] = data['text'].apply(text_to_sequence)
data['mel_spectrogram'] = data['path'].apply(lambda x: load_and_convert_to_mel(x))

# Padding for text data and mel spectrograms
max_length = 100
text_sequences = pad_sequences(data['text_encoded'], maxlen=max_length, padding='post')
mels = list(data['mel_spectrogram'])
padded_mels = pad_mel_specs(mels, max_length=max_length)

# Add a new dimension to text sequences for the model
text_sequences = np.expand_dims(text_sequences, -1)

# Split data into training and test sets
train_text, test_text, train_mel, test_mel = train_test_split(text_sequences, padded_mels, test_size=0.2)

# Build the model
def build_model(input_dim, timesteps, output_dim):
    input_layer = Input(shape=(timesteps, input_dim))
    x = Conv1D(128, kernel_size=5, strides=1, padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(128, kernel_size=5, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = LSTM(256, return_sequences=True)(x)
    x = TimeDistributed(Dense(output_dim))(x)
    model = Model(inputs=input_layer, outputs=x)
    return model

# Define input and output dimensions
input_dim = train_text.shape[2]  # Features per timestep in text sequence
timesteps = train_text.shape[1]  # Length of text sequence
output_dim = train_mel.shape[2]  # Features per timestep in mel spectrogram

# Create and compile the model
model = build_model(input_dim, timesteps, output_dim)
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(train_text, train_mel, validation_data=(test_text, test_mel), epochs=200, batch_size=2)

Model: "functional_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 100, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_34 (Conv1D)              │ (None, 100, 128)       │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 100, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_36 (ReLU)                 │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_35 (Conv1D)              │ (None, 100, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 100, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_37 (ReLU)                 │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_20 (LSTM)                  │ (None, 100, 256)       │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_22             │ (None, 100, 80)        │        20,560 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 498,640 (1.90 MB)

 Trainable params: 498,128 (1.90 MB)

 Non-trainable params: 512 (2.00 KB)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - loss: 2889.3650 - val_loss: 3654.8225
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 2869.2163 - val_loss: 3634.5525
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 2847.7307 - val_loss: 3613.6909
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - loss: 2823.2927 - val_loss: 3587.5212
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - loss: 2797.3926 - val_loss: 3552.3750
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - loss: 2771.4338 - val_loss: 3519.1934
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - loss: 2744.6174 - val_loss: 3483.3950
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step - loss: 2716.4263 - val_loss: 3447.9155
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - loss: 2686.7976 - val_loss: 3410.9893
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - loss: 2655.8291 - val_loss: 3374.8293
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - loss: 2623.9199 - val_loss: 3339.197

In [45]:
data

,path,text,text_encoded,mel_spectrogram
0,hack\ans10001_982944754.wav,"Денлоны яулап алганнан соң, Англиядә озак елла...","[1044, 1077, 1085, 1083, 1086, 1085, 1099, 32,...","[[-59.391212, -45.87086, -40.448845, -40.26476..."
1,hack\ans10003_184648589.wav,Моның аркасында ачылышы утыз биш миллиметрлы н...,"[1052, 1086, 1085, 1099, 1187, 32, 1072, 1088,...","[[-80.0, -80.0, -79.93655, -64.24773, -61.0934..."
2,hack\ans10003_724102486.wav,Моның аркасында ачылышы утыз биш миллиметрлы н...,"[1052, 1086, 1085, 1099, 1187, 32, 1072, 1088,...","[[-70.6089, -63.285088, -53.973717, -51.55567,..."
3,hack\ans10005_724102486.wav,Се шулай ук фотогеник Ма харизмадан бигрәк тыш...,"[1057, 1077, 32, 1096, 1091, 1083, 1072, 1081,...","[[-72.502174, -62.106377, -55.958374, -51.6086..."
4,hack\ans10006_724102486.wav,Сайлаулар вакытында Се Маның кризис булганда и...,"[1057, 1072, 1081, 1083, 1072, 1091, 1083, 107...","[[-70.790054, -64.08453, -55.762535, -49.15542..."
5,hack\ans10007_227122305.wav,Яңа катнашма алу өчен ике коры порошокны бер с...,"[1071, 1187, 1072, 32, 1082, 1072, 1090, 1085,...","[[-80.0, -80.0, -80.0, -80.0, -78.082695, -67...."
6,hack\ans10007_925918476.wav,Яңа катнашма алу өчен ике коры порошокны бер с...,"[1071, 1187, 1072, 32, 1082, 1072, 1090, 1085,...","[[-80.0, -73.234314, -64.66592, -63.40349, -43..."
7,hack\ans10011_495807986.wav,"Хакимият хәбәр иткәнчә, еллык бәйрәмгә багышла...","[1061, 1072, 1082, 1080, 1084, 1080, 1103, 109...","[[-80.0, -80.0, -80.0, -80.0, -80.0, -57.67504..."
8,hack\ans10012_1432774430.wav,"Күз белән күрү, яки күрү сәләте, күрү системас...","[1050, 1199, 1079, 32, 1073, 1077, 1083, 1241,...","[[-70.86417, -50.113895, -46.32516, -47.48358,..."
9,hack\ans10015_464206186.wav,"Без моны кабатлаган саен, минем күңелем чын-чы...","[1041, 1077, 1079, 32, 1084, 1086, 1085, 1099,...","[[-75.05032, -68.106834, -68.7334, -62.100452,..."


In [46]:
import numpy as np
import librosa
import soundfile as sf
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text_to_audio(model, input_text, max_length=100, sample_rate=22050, n_fft=2048, hop_length=512):
    # Convert input text to sequence
    text_sequence = np.array([ord(char) for char in input_text])
    text_sequence = pad_sequences([text_sequence], maxlen=max_length, padding='post')
    text_sequence = np.expand_dims(text_sequence, -1)  # Match the input shape of the model

    # Predict mel spectrogram
    predicted_mel = model.predict(text_sequence)
    predicted_mel = predicted_mel.squeeze()  # Remove batch dimension

    # Convert mel spectrogram to audio using Griffin-Lim
    # Transform dB to power
    mel_spec_power = librosa.db_to_power(predicted_mel)
    # Inverse mel to stft
    stft_matrix = librosa.feature.inverse.mel_to_stft(mel_spec_power, sr=sample_rate, n_fft=n_fft)
    # Griffin-Lim algorithm to get the audio time series
    audio_time_series = librosa.griffinlim(stft_matrix, hop_length=hop_length, n_iter=32)

    # Save or play audio
    output_file = 'output_voice_message.wav'
    sf.write(output_file, audio_time_series, sample_rate)
    print(f"Audio output saved to {output_file}")

    return output_file

# Assuming `model` is your trained Keras model
text_input = "Денлоны яулап алганнан соң, Англиядә озак елларга тынычлык урнаша."
audio_output = text_to_audio(model, text_input)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Audio output saved to output_voice_message.wav


In [1]:
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, LSTM, Dense, TimeDistributed, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load audio and convert to mel-spectrogram
def load_and_convert_to_mel(path, n_mels=80, n_fft=2048, hop_length=512, sample_rate=22050):
    audio, sr = librosa.load(path, sr=sample_rate)
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

# Convert text to sequence of character ordinals
def text_to_sequence(text):
    return [ord(char) for char in text]

# Pad mel spectrograms to ensure consistent dimensions
def pad_mel_specs(mel_specs, max_length=100):
    n_mels = mel_specs[0].shape[0]
    padded_mels = np.zeros((len(mel_specs), max_length, n_mels))  # Changed dimensions
    for i, mel_spec in enumerate(mel_specs):
        if mel_spec.shape[1] > max_length:
            padded_mels[i, :max_length, :] = mel_spec[:, :max_length].T
        else:
            padded_mels[i, :mel_spec.shape[1], :] = mel_spec.T
    return padded_mels

# Load data
data = pd.read_csv('hack/dataset.csv')[:10]

# Process text and audio to sequences and mel spectrograms
data['text_encoded'] = data['text'].apply(text_to_sequence)
data['mel_spectrogram'] = data['path'].apply(lambda x: load_and_convert_to_mel(x))

# Padding for text data and mel spectrograms
max_length = 100
text_sequences = pad_sequences(data['text_encoded'], maxlen=max_length, padding='post')
mels = list(data['mel_spectrogram'])
padded_mels = pad_mel_specs(mels, max_length=max_length)

# Add a new dimension to text sequences for the model
text_sequences = np.expand_dims(text_sequences, -1)

# Split data into training and test sets
train_text, test_text, train_mel, test_mel = train_test_split(text_sequences, padded_mels, test_size=0.2)

# Define the improved model function
def build_improved_model(input_dim, timesteps, output_dim):
    input_layer = Input(shape=(timesteps, input_dim))
    x = Conv1D(128, kernel_size=5, strides=1, padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.25)(x)  # Adding dropout
    x = Conv1D(128, kernel_size=5, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.25)(x)  # Adding dropout
    x = LSTM(256, return_sequences=True)(x)
    x = TimeDistributed(Dense(output_dim))(x)
    model = Model(inputs=input_layer, outputs=x)
    return model

# Define input and output dimensions
input_dim = train_text.shape[2]
timesteps = train_text.shape[1]
output_dim = train_mel.shape[2]

# Create and compile the model
model = build_improved_model(input_dim, timesteps, output_dim)
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')

# Callbacks for early stopping and reducing learning rate on plateau
callbacks = [
    EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
]

# Training with callbacks
history = model.fit(
    train_text, 
    train_mel, 
    validation_data=(test_text, test_mel), 
    epochs=20000, 
    batch_size=32,  # Increased batch size
    callbacks=callbacks
)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 100, 128)       │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 100, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 100, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100, 256)       │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 100, 80)        │        20,560 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 498,640 (1.90 MB)

 Trainable params: 498,128 (1.90 MB)

 Non-trainable params: 512 (2.00 KB)

Epoch 1/20000
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - loss: 3081.5483 - val_loss: 2914.5779 - learning_rate: 0.0010
Epoch 2/20000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - loss: 3062.2681 - val_loss: 2891.6096 - learning_rate: 0.0010
Epoch 3/20000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - loss: 3041.4175 - val_loss: 2866.2700 - learning_rate: 0.0010
Epoch 4/20000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - loss: 3015.8799 - val_loss: 2839.2913 - learning_rate: 0.0010
Epoch 5/20000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - loss: 2986.3975 - val_loss: 2806.7969 - learning_rate: 0.0010
Epoch 6/20000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - loss: 2955.6807 - val_loss: 2772.6394 - learning_rate: 0.0010
Epoch 7/20000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - loss: 2925.6938 - val_loss: 2738.9988 - learning_rate: 0.0010
Epoch 8/20000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - loss: 2895.3145 - val_loss: 2702.8525 - learning_rate: 0.0010
Epoch 9/20000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - loss: 2863

In [53]:
import numpy as np
import librosa
import soundfile as sf
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text_to_audio(model, input_text, max_length=100, sample_rate=22050, n_fft=2048, hop_length=512):
    # Convert input text to sequence
    text_sequence = np.array([ord(char) for char in input_text])
    text_sequence = pad_sequences([text_sequence], maxlen=max_length, padding='post')
    text_sequence = np.expand_dims(text_sequence, -1)  # Match the input shape of the model

    # Predict mel spectrogram
    predicted_mel = model.predict(text_sequence)
    predicted_mel = predicted_mel.squeeze()  # Remove batch dimension

    # Convert mel spectrogram to audio using Griffin-Lim
    # Transform dB to power
    mel_spec_power = librosa.db_to_power(predicted_mel)
    # Inverse mel to stft
    stft_matrix = librosa.feature.inverse.mel_to_stft(mel_spec_power, sr=sample_rate, n_fft=n_fft)
    # Griffin-Lim algorithm to get the audio time series
    audio_time_series = librosa.griffinlim(stft_matrix, hop_length=hop_length, n_iter=32)

    # Save or play audio
    output_file = 'output_voice_message.wav'
    sf.write(output_file, audio_time_series, sample_rate)
    print(f"Audio output saved to {output_file}")

    return output_file

# Assuming `model` is your trained Keras model
text_input = "Денлоны яулап алганнан соң, Англиядә озак елларга тынычлык урнаша."
audio_output = text_to_audio(model, text_input)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Audio output saved to output_voice_message.wav


In [2]:
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, LSTM, Dense, TimeDistributed, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load audio and convert to mel-spectrogram
def load_and_convert_to_mel(path, n_mels=80, n_fft=2048, hop_length=512, sample_rate=22050):
    audio, sr = librosa.load(path, sr=sample_rate)
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

# Convert text to sequence of character ordinals
def text_to_sequence(text):
    return [ord(char) for char in text]

# Pad mel spectrograms to ensure consistent dimensions
def pad_mel_specs(mel_specs, max_length=100):
    n_mels = mel_specs[0].shape[0]
    padded_mels = np.zeros((len(mel_specs), max_length, n_mels))  # Changed dimensions to match input
    for i, mel_spec in enumerate(mel_specs):
        if mel_spec.shape[1] > max_length:
            padded_mels[i, :max_length, :] = mel_spec[:, :max_length].T
        else:
            padded_mels[i, :mel_spec.shape[1], :] = mel_spec.T
    return padded_mels

# Load data
data = pd.read_csv('hack/dataset.csv')

# Process text and audio to sequences and mel spectrograms
data['text_encoded'] = data['text'].apply(text_to_sequence)
data['mel_spectrogram'] = data['path'].apply(lambda x: load_and_convert_to_mel(x))

# Padding for text data and mel spectrograms
max_length = 100
text_sequences = pad_sequences(data['text_encoded'], maxlen=max_length, padding='post')
mels = list(data['mel_spectrogram'])
padded_mels = pad_mel_specs(mels, max_length=max_length)

# Add a new dimension to text sequences for the model
text_sequences = np.expand_dims(text_sequences, -1)

# Split data into training and test sets
train_text, test_text, train_mel, test_mel = train_test_split(text_sequences, padded_mels, test_size=0.2)

# Define the improved model function with MAE metric
def build_improved_model(input_dim, timesteps, output_dim):
    input_layer = Input(shape=(timesteps, input_dim))
    x = Conv1D(128, kernel_size=5, strides=1, padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.25)(x)
    x = Conv1D(128, kernel_size=5, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.25)(x)
    x = LSTM(256, return_sequences=True)(x)
    x = TimeDistributed(Dense(output_dim))(x)
    model = Model(inputs=input_layer, outputs=x)
    return model

# Define input and output dimensions
input_dim = train_text.shape[2]
timesteps = train_text.shape[1]
output_dim = train_mel.shape[2]

# Set learning rate as a variable
learning_rate = 0.001

# Create and compile the model with MAE as a metric
model = build_improved_model(input_dim, timesteps, output_dim)
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])
model.summary()

# Callbacks for early stopping (monitor 'val_mean_absolute_error') and reducing learning rate on plateau
callbacks = [
    EarlyStopping(monitor='val_mean_absolute_error', patience=5, verbose=1, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_mean_absolute_error', factor=0.1, patience=3, verbose=1)
]

# Training with callbacks, tracking loss and MAE
history = model.fit(
    train_text, 
    train_mel, 
    validation_data=(test_text, test_mel), 
    epochs=500, 
    batch_size=2,  # Increased batch size
    verbose=1  # Ensures output logs for each epoch
)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 100, 128)       │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 100, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 100, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100, 256)       │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 100, 80)        │        20,560 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 498,640 (1.90 MB)

 Trainable params: 498,128 (1.90 MB)

 Non-trainable params: 512 (2.00 KB)

Epoch 1/500
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 131s 85ms/step - loss: 809.8647 - mean_absolute_error: 22.0080 - val_loss: 253.8761 - val_mean_absolute_error: 12.6541
Epoch 2/500
 313/1453 ━━━━━━━━━━━━━━━━━━━━ 1:41 89ms/step - loss: 251.3495 - mean_absolute_error: 12.6324

KeyboardInterrupt: 

In [55]:
import numpy as np
import librosa
import soundfile as sf
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text_to_audio(model, input_text, max_length=100, sample_rate=22050, n_fft=2048, hop_length=512):
    # Convert input text to sequence
    text_sequence = np.array([ord(char) for char in input_text])
    text_sequence = pad_sequences([text_sequence], maxlen=max_length, padding='post')
    text_sequence = np.expand_dims(text_sequence, -1)  # Match the input shape of the model

    # Predict mel spectrogram
    predicted_mel = model.predict(text_sequence)
    predicted_mel = predicted_mel.squeeze()  # Remove batch dimension

    # Convert mel spectrogram to audio using Griffin-Lim
    # Transform dB to power
    mel_spec_power = librosa.db_to_power(predicted_mel)
    # Inverse mel to stft
    stft_matrix = librosa.feature.inverse.mel_to_stft(mel_spec_power, sr=sample_rate, n_fft=n_fft)
    # Griffin-Lim algorithm to get the audio time series
    audio_time_series = librosa.griffinlim(stft_matrix, hop_length=hop_length, n_iter=32)

    # Save or play audio
    output_file = 'output_voice_message.wav'
    sf.write(output_file, audio_time_series, sample_rate)
    print(f"Audio output saved to {output_file}")

    return output_file

# Assuming `model` is your trained Keras model
text_input = "Денлоны яулап алганнан соң, Англиядә озак елларга тынычлык урнаша."
audio_output = text_to_audio(model, text_input)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Audio output saved to output_voice_message.wav
